In [1]:
import pandas as pd
import os
from datetime import datetime
import string

folderpath_CuringAct = r"C:\Users\Ployphet.veerasak\Desktop\Defect\Liner air\Curing actual\RawData" #os.path.join(os.getcwd(), "RawData")
file_CuringAct = [fCA for fCA in os.listdir(folderpath_CuringAct) if fCA.endswith(".xlsx")]

df_list = []

for CAmo in file_CuringAct:
    try:
        month_year = CAmo.split('.')[0]  
        date_obj = datetime.strptime(month_year, "%B %Y")  
    except ValueError:
        print(f"Skipping file {CAmo}: Invalid filename format")
        continue
    
    filepath_CuringAct = os.path.join(folderpath_CuringAct, CAmo)
    dfCuringAct = pd.read_excel(filepath_CuringAct)

    dfCuringAct = dfCuringAct.dropna(subset=["PCode"])

    dfCuringAct = dfCuringAct.drop(columns=["Item", "S.Code", "Tire Size"], errors="ignore")

    dfCuringAct = dfCuringAct.rename(columns={"PCode": "Date"})

    dfCuringAct.columns = dfCuringAct.columns.astype(str)
    dfCuringAct = dfCuringAct.loc[:, ~dfCuringAct.columns.str.contains("^Unnamed")]

    new_columns = {"Date": "Date"}  
    for col in dfCuringAct.columns[1:]:  
        if col.isdigit():  
            new_col_name = f"{col}/{date_obj.strftime('%m/%Y')}"  
            new_columns[col] = new_col_name

    dfCuringAct = dfCuringAct.rename(columns=new_columns)

    df_list.append(dfCuringAct)
    
df_merged = pd.concat(df_list, ignore_index = True).fillna(0)

df_merged = df_merged.T

df_merged.columns = df_merged.iloc[0]  
df_merged = df_merged.iloc[1:]  

df_merged = df_merged.groupby(df_merged.columns, axis=1).sum()

df_merged.reset_index(inplace=True)
df_merged.rename(columns={"index": "Date"}, inplace=True)

df_merged.insert(0, "Date", df_merged.pop("Date"))

df_merged["Date"] = pd.to_datetime(df_merged["Date"], format="%d/%m/%Y")

df_merged = df_merged.melt(id_vars=["Date"], var_name="Size", value_name="Actual")

df_merged = df_merged.sort_values(by=["Date"], ascending = True)

df_merged["Week"] = df_merged["Date"].dt.day.apply(
    lambda d: "Week 1" if d < 8 else
    "Week 2" if d < 15 else
    "Week 3" if d < 22 else
    "Week 4"
)

cols = df_merged.columns.tolist()
cols.insert(1, cols.pop(cols.index("Week")))
df_merged = df_merged[cols]

df_merged.to_csv("CuringActual.csv", index = False)

C:\Users\Ployphet.veerasak\AppData\Local\Temp\ipykernel_16988\3388132052.py:48: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_merged.groupby(df_merged.columns, axis=1).sum()
C:\Users\Ployphet.veerasak\AppData\Local\Temp\ipykernel_16988\3388132052.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_merged.reset_index(inplace=True)
C:\Users\Ployphet.veerasak\AppData\Local\Temp\ipykernel_16988\3388132052.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.cop